

# Título   Chequear ternaria

## Autor: Jose Chelquer
## Fecha de última modificación: 22/10/2025
## Descripción:

Chequear en un número de filas si fueron bien clasificadas en clase_ternaria

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [ ]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive

casos_a_chequear=1000     # cantidad de filas a chequear

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [ ]:
dataset_path = '~/buckets/b1/datasets'
dataset_file = 'competencia_02.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [ ]:
#output_file='competencia_02.csv.gz'


In [ ]:
## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [ ]:
import os
import pandas as pd

In [ ]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# leer el dataset

file_path = os.path.join(dataset_path, dataset_file)
print (f'Leyendo {file_path}')
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.shape

Leyendo ~/buckets/b1/datasets/competencia_02_crudo.csv.gz
   numero_de_cliente  foto_mes  active_quarter  cliente_vip  internet  \
0          249221109    201901               1            0         1   
1          249221468    201901               1            0         1   
2          249223005    201901               1            0         1   
3          249228180    201901               1            0         1   
4          249232117    201901               1            0         1   

   cliente_edad  cliente_antiguedad  mrentabilidad  mrentabilidad_annual  \
0            59                 276        7597.55              47433.58   
1            51                  13         738.16               -325.09   
2            46                 178        1014.31               9434.15   
3            64                 296        1028.33               6873.80   
4            77                 349       11617.61              43291.55   

   mcomisiones  ...  Visa_madelantodolares  Vi

In [ ]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes, meses):
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm


In [ ]:
columnas_interes = ['numero_de_cliente', 'foto_mes', 'clase_ternaria']

# Obtener una muestra aleatoria (puedes especificar el tamaño de la muestra con n)
df=df[columnas_de_interes]
muestra = df.sample(n=casos_a_chequear)
muestra['clase_ternara'].value_counts()

In [ ]:
def chequear_fila (fila, df):
  clase_ternaria=fila['clase_ternaria']
  numero_de_cliente=fila['numero_de_cliente']
  mes=fila['foto_mes']
  mes_mas_1=sig_mes(mes, 1)
  mes_mas_2=sig_mes(mes, 2)
  existe1=df[(df['numero_de_cliente']==numero_de_cliente) & (df['foto_mes']==mes_mas_1)].shape[0]>0
  existe2=df[(df['numero_de_cliente']==numero_de_cliente) & (df['foto_mes']==mes_mas_2)].shape[0]>0
  if clase_ternaria=='CONTINUA':
    ok=existe1 and existe2
  elif clase_ternaria=='BAJA+1':
    ok=not existe1
  elif clase_ternaria=='BAJA+2':
    ok=existe1 and not existe2
  else:
    ok=False
  mensaje=f'Clte: {numero_de_cliente} {clase_ternaria} ok: {ok}   ex1: {existe1} ex2: {existe2}'
  return (ok, mensaje)

Detectando existe1 y existe2


In [ ]:
num=0
errores=0
for index, muestra in muestra.iterrows():
  num=num+1
  print (f'Chequeando fila {num}')
  ok, mensaje=chequear_fila(muestra, df)
  print (num, mensaje)
  if not ok:
    errores=errores+1


Creando clase ternaria


In [ ]:
errores

In [ ]:
# Eliminar los campos creados
print (f'Eliminando campos creados')
df = df.drop(columns=['cli_mes', 'cli_mesMas1', 'cli_mesMas2', 'existe1', 'existe2'])

Eliminando campos creados


In [ ]:
ruta_nuevo_archivo = os.path.join(dataset_path, output_file)
# Guarda el DataFrame en el nuevo archivo
print (f'Guardando el archivo de salida')
df.to_csv(ruta_nuevo_archivo, index=False, compression='gzip')
print(f"El archivo ha sido guardado en: {ruta_nuevo_archivo}")

Guardando el archivo de salida
El archivo ha sido guardado en: ~/buckets/b1/datasets/competencia_02.csv.gz


In [ ]:
df['clase_ternaria'].value_counts()

clase_ternaria
CONTINUA    3768977
BAJA+1       486920
BAJA+2       479696
Name: count, dtype: int64

In [ ]:
df.shape[0]

4735593